In [ ]:
from __future__ import annotations

import os
import json
import uuid
import datetime as dt
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple, Literal

[Cell 1] 설정/의존성/환경변수

OpenAI/KT API 키 로드(환경변수)

In [ ]:
# =========================
# Cell 1) 환경/설정/공통 유틸
# - 1일차 목표: "LLM 없이도" 등급/Top5/누락 후보까지 뽑히는 뼈대 준비
# - 규칙: 앞으로 모든 함수명은 report_로 시작
# =========================

# (선택) 데이터 검증/스키마 강제용
try:
    from pydantic import BaseModel, Field
    PydanticAvailable = True
except Exception:
    PydanticAvailable = False


# -------------------------
# 1) 전역 설정(난이도 낮춘 MVP용)
# -------------------------

# 등급 체계(임시): 필요하면 팀 룰에 맞게 바꾸면 됨
RiskGrade = Literal["Low", "Med", "High", "Critical"]

# 점수 테이블(임시)
REPORT_SCORE_TABLE = {
    "quant_status": {  # 정량 항목 상태
        "OK": 0,
        "WARN": 2,
        "FAIL": 5,
        "MISSING": 4,
        "UNCERTAIN": 2,  # 단위/기간 불명 등
    },
    "qual_answer": {   # 정성 문항 응답
        "YES": 0,
        "PARTIAL": 2,
        "NO": 4,
        "MISSING": 4,
        "UNCERTAIN": 2,
    },
}

# 등급 산정 구간(임시)
# total_score가 어느 구간인지에 따라 grade 결정
REPORT_GRADE_BINS: List[Tuple[int, int, RiskGrade]] = [
    (0, 5, "Low"),
    (6, 12, "Med"),
    (13, 20, "High"),
    (21, 10**9, "Critical"),
]

# Top5 개수 고정
REPORT_TOPK = 5


# -------------------------
# 2) 공통 유틸 함수
# -------------------------

def report_now_iso() -> str:
    """현재 시간을 ISO 문자열로 반환 (로그/추적용)."""
    return dt.datetime.now(dt.timezone.utc).isoformat()

def report_new_request_id(prefix: str = "req") -> str:
    """요청 추적용 request_id 생성."""
    return f"{prefix}_{uuid.uuid4().hex[:12]}"

def report_safe_get(d: Dict[str, Any], key: str, default: Any = None) -> Any:
    """dict 안전 접근."""
    if not isinstance(d, dict):
        return default
    return d.get(key, default)

def report_normalize_text(x: Any) -> str:
    """텍스트 필드 안전 정규화."""
    if x is None:
        return ""
    return str(x).strip()

def report_to_float(x: Any) -> Optional[float]:
    """숫자 변환 유틸. 실패 시 None."""
    try:
        if x is None or x == "":
            return None
        return float(x)
    except Exception:
        return None

def report_clip_list(items: List[Any], max_len: int = 3) -> List[Any]:
    """리스트 길이 제한(근거 스니펫 과다 방지)."""
    if not isinstance(items, list):
        return []
    return items[:max_len]


# -------------------------
# 3) (선택) Pydantic 스키마: 1일차 중간 산출물용
#    - pydantic이 없으면 dict로만 진행 가능
# -------------------------

if PydanticAvailable:
    class ReportTopRiskCandidate(BaseModel):
        item_id: str
        title: str
        impact_area: Literal["E", "S", "G", "Common"] = "Common"
        score: int = 0
        evidence_snippets: List[str] = Field(default_factory=list)

    class ReportMissingCandidate(BaseModel):
        item_id: str
        type: Literal["missing", "uncertain"]
        why_needed: str
        what_to_submit: str
        severity: Literal["High", "Med", "Low"] = "Med"

    class ReportScoringResult(BaseModel):
        request_id: str
        generated_at: str
        risk_grade: RiskGrade
        total_score: int
        contributions: List[Dict[str, Any]] = Field(default_factory=list)
        top5_candidates: List[ReportTopRiskCandidate] = Field(default_factory=list)
        missing_or_uncertain_candidates: List[ReportMissingCandidate] = Field(default_factory=list)

else:
    # pydantic이 없을 때도 동작하도록 placeholder 타입(문서용)
    ReportTopRiskCandidate = dict
    ReportMissingCandidate = dict
    ReportScoringResult = dict


# -------------------------
# 4) (디버그) 환경 확인
# -------------------------

print("PydanticAvailable:", PydanticAvailable)
print("REPORT_TOPK:", REPORT_TOPK)
print("REPORT_GRADE_BINS:", REPORT_GRADE_BINS)
print("Loaded at:", report_now_iso())


PydanticAvailable: True
REPORT_TOPK: 5
REPORT_GRADE_BINS: [(0, 5, 'Low'), (6, 12, 'Med'), (13, 20, 'High'), (21, 1000000000, 'Critical')]
Loaded at: 2026-01-21T00:53:38.458830+00:00
